In [2]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import sys
import s3fs
import yaml
import numpy as np
import fireducks.pandas as pd
import warnings
from dotenv import load_dotenv
from tqdm import tqdm
from pprint import pprint
import joblib

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import (
                                    cross_val_score,
                                    RandomizedSearchCV,
                                    GridSearchCV, 
                                    train_test_split,
                                    StratifiedKFold
                                    )
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from xgboost import XGBClassifier
from gensim.models import Word2Vec, FastText

sys.path.append("../src")
from ml_utils import *

In [4]:
load_dotenv()
pd.set_option("display.max_columns", None)
warnings.simplefilter("ignore")
fs = s3fs.S3FileSystem(
            client_kwargs={"endpoint_url": "https://minio.lab.sspcloud.fr"},
            key=os.environ["Accesskey"],
            secret=os.environ["Secretkey"],
            token=os.environ["Token"]
)

# Data Preprocessing

In [6]:
with fs.open("elissamim/text_classification_men/data/stages-votes.json", "r") as file:
    df = pd.read_json(file)

df = df.groupby("phrase_text", as_index = False)["sol"].apply(lambda x: x.mode().iloc[0])
df["sol"]=df["sol"].apply(lambda x: 1 if x == "ok" else 0)
df["clean_phrase_text"] = df["phrase_text"].apply(lambda x: nltk_text_preprocessing(x, True))
df = df[df["clean_phrase_text"] != ""]
df.head()

,phrase_text,sol,clean_phrase_text
0,* Aider à la mise en place de l évènement Shar...,0,aider mise place évènemer shareplan envoi rapp...
1,* Comprendre le métier des achats * Comment or...,0,comprendre métier achat comment organiser appe...
2,* Fendre du bois en forêt au merlin manuelleme...,0,fendre boi forêt merlin manuellemer débarder b...
4,"2 jours au CDI , 1 jour en arts plastiques , 1...",0,2 jour cdi 1 jour art plastique 1 jour musiqu ...
5,4 jours au sein du Bureau des affaires institu...,1,4 jour sein bureau affaire institutionnel fina...


# Model selection (static embedding (sparse or dense) + classification algorithm) with grid search and cross validation

In [8]:
# Separate validation data for later
X = df["clean_phrase_text"]
y = df["sol"]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size= 0.2,
                                                    stratify=y,
                                                    random_state = 42)

tokenized_texts = [text.split() for text in X_train]
word2vec_model = Word2Vec(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)
fasttext_model = FastText(sentences = tokenized_texts,
                         vector_size = 100,
                         window = 5,
                         min_count = 1,
                         workers = 4,
                         seed = 42)

static_embedding_models = {
    # Sparse embeddings
    "Bag of Words":CountVectorizer(),
    "TF":TfidfVectorizer(use_idf=False),
    "TF-IDF":TfidfVectorizer(),
    # Dense embeddings
    "Word2Vec": MeanEmbeddingVectorizer(model=word2vec_model),
    "FastText": MeanEmbeddingVectorizer(model=fasttext_model)
}

classification_models = {
    "Logistic Regression":LogisticRegression(),
    "Random Forest":RandomForestClassifier(),
    "Linear SVM":SVC(kernel="linear", probability=True),
    "Multinomial Naive Bayes":MultinomialNB(),
    "XGBoost":XGBClassifier(use_label_encoder=False, eval_metric="logloss")
}

# Import hyperparameter grid for tuning models
with open("../configs/ml_hyperparameters.yaml", "r", encoding = "utf-8") as file:
    params_grid = yaml.load(file, Loader = yaml.FullLoader)

In [15]:
dict_scores, dict_params = {}, {}

for embedding_name, embedding_model in tqdm(static_embedding_models.items(),
                                           desc="Static embeddings"):

    dict_scores[embedding_name], dict_params[embedding_name] = {}, {}
    
    for classification_name, classification_model in tqdm(classification_models.items(),
                                                         desc="Classification algorithms"):

        # Multinomial NB is not suited for dense vectors
        if embedding_name in ["Word2Vec", "FastText"] and classification_name == "Multinomial Naive Bayes":
            continue

        steps = [("feature_extraction", embedding_model)]

        # For Logistic Regression and Linear SVM, and for dense embeddings, add standardisation
        if embedding_name in ["Word2Vec", "FastText"] and classification_name in ["Logistic Regression", "Linear SVM"]:
            steps.append(("standardisation", StandardScaler()))

        steps.append(("classifier", classification_model))

        pipeline = Pipeline(steps)

        # We compute scores using Grid Search on the parameter grid to do model selection with best
        # hyperparameters
        cv = StratifiedKFold(n_splits=5, 
                             shuffle = True, 
                             random_state=42)
        
        if ((embedding_name in params_grid) or (classification_name in params_grid)) and (embedding_name != "FastText"):

            selected_params_grid = {
                **params_grid.get(embedding_name, {}),
                **params_grid.get(classification_name, {})
            }

            grid_search = RandomizedSearchCV(
                pipeline,
                param_distributions=selected_params_grid,
                cv=cv,
                scoring="accuracy",
                n_jobs=-1,
                verbose=0,
                n_iter = 5
            )

            grid_search.fit(X_train, y_train)
            
            score = grid_search.best_score_
            dict_params[embedding_name][classification_name] = grid_search.best_params_

        else:

            scores = cross_val_score(
                pipeline,
                X_train, 
                y_train,
                cv=cv,
                scoring="accuracy"
            )

            score = np.mean(scores)
            
        dict_scores[embedding_name][classification_name] = score

Classification algorithms:  80%|████████  | 4/5 [00:13<00:02,  2.29s/it]/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:47:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:47:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:47:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:47:35] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/si

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  D


Classification algorithms:  20%|██        | 1/5 [00:53<03:35, 53.88s/it]

✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


...


Classification algorithms:  40%|████      | 2/5 [01:44<02:36, 52.07s/it][nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/onyxia/nltk_data..

  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
  Using cached https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the packag


Classification algorithms:  60%|██████    | 3/5 [07:59<06:38, 199.48s/it]/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:55:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:55:50] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:55:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:55:51] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--Collecting fr-core-news-sm==3.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--Collecting fr-core-news-sm==3.8.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--  Downloading https://github.com/explosion/spacy-models/releases/download/fr_core_news_sm-3.8.0/fr_core_news_sm-3.8.0-py3-none-any.whl (16.3 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/16.3 MB ? eta -:--:--

/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:56:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:56:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:56:07] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:56:08] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/usr/local/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [09:56:08] WARNING: /w

In [16]:
pprint(dict_scores)

{'Bag of Words': {'Linear SVM': 0.6433962264150944,
                  'Logistic Regression': 0.6754716981132074,
                  'Multinomial Naive Bayes': 0.619811320754717,
                  'Random Forest': 0.6839622641509433,
                  'XGBoost': 0.6735849056603772},
 'FastText': {'Linear SVM': 0.6075471698113207,
              'Logistic Regression': 0.590566037735849,
              'Random Forest': 0.5999999999999999,
              'XGBoost': 0.5877358490566038},
 'TF': {'Linear SVM': 0.6424528301886793,
        'Logistic Regression': 0.6443396226415095,
        'Multinomial Naive Bayes': 0.6584905660377358,
        'Random Forest': 0.6537735849056603,
        'XGBoost': 0.620754716981132},
 'TF-IDF': {'Linear SVM': 0.6264150943396226,
            'Logistic Regression': 0.6641509433962264,
            'Multinomial Naive Bayes': 0.6575471698113208,
            'Random Forest': 0.6981132075471699,
            'XGBoost': 0.6641509433962265},
 'Word2Vec': {'Linear SVM': 0.60

In [20]:
best_params = dict_params["TF-IDF"]["Random Forest"]
best_params

{'feature_extraction__sublinear_tf': True,
 'feature_extraction__norm': None,
 'feature_extraction__ngram_range': (1, 2),
 'feature_extraction__min_df': 0.001,
 'feature_extraction__max_features': 1000,
 'feature_extraction__max_df': 0.8,
 'classifier__n_estimators': 500,
 'classifier__min_samples_split': 10,
 'classifier__min_samples_leaf': 2,
 'classifier__max_features': 'sqrt',
 'classifier__max_depth': None,
 'classifier__bootstrap': True}

✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


# Model evaluation

In [26]:
pipeline = Pipeline([
    ("feature_extraction", TfidfVectorizer()),
    ("classifier", RandomForestClassifier())
])
pipeline.set_params(**best_params)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))
confusion_matrix(y_test, y_pred)

0.6654135338345865
              precision    recall  f1-score   support

           0       0.69      0.80      0.74       161
           1       0.60      0.46      0.52       105

    accuracy                           0.67       266
   macro avg       0.65      0.63      0.63       266
weighted avg       0.66      0.67      0.65       266



array([[129,  32],
       [ 57,  48]])

In [27]:
joblib.dump(pipeline,
            "../models/ml_model.joblib")

['../models/ml_model.joblib']